In [1]:
import math
import numpy as np
import scipy.stats as stats
from scipy.stats import norm
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportions_ztest
from tqdm.notebook import tqdm

### Вариант 1

In [2]:
# Evan's Awesome A/B Tools - https://www.evanmiller.org/ab-testing/sample-size.html

In [3]:
# https://www.evanmiller.org/ab-testing/sample-size.html#!10;80;5;1;0

In [4]:
# Результат - 14313

### Вариант 2

In [5]:
n = 10000
res = []
for _ in tqdm(range(n)):

  p1 = np.random.binomial(1,0.10,14313)
  p2 = np.random.binomial(1,0.11,14313)

  cnts = [p1.sum(),p2.sum()]
  nobs = [len(p1),len(p2)]
  pval = proportions_ztest(cnts,nobs)[1]
  res.append(pval <= 0.05)
np.mean(res)

  0%|          | 0/10000 [00:00<?, ?it/s]

0.7898

### Вариант 3

In [6]:
baseline_rate = 0.1
practical_significance = 0.01
confidence_level = 0.05 
sensitivity = 0.8 

effect_size = sms.proportion_effectsize(baseline_rate, baseline_rate + practical_significance)
sample_size = sms.NormalIndPower().solve_power(effect_size = effect_size, power = sensitivity, 
                                               alpha = confidence_level, ratio=1)
print(sample_size)

14744.104836925611


### Вариант 4

In [7]:
def get_sample_size(p1,p2,alpha=0.05,power=0.8,r=1):
    
    p_bar = (p1 + r*p2)/(r+1)
    q_bar = 1 - p_bar
    s = np.abs(p1-p2)
    
    q1 = 1 - p1
    q2 = 1 - p2
    
    z_alpha = norm.ppf(1 - alpha/2)
    z_beta = norm.ppf(power)
    
    m = ((z_alpha * np.sqrt((r+1) * p_bar * q_bar) +
        z_beta * np.sqrt(r*p1*q1 + p2*q2))**2/
        (r*s**2))
    
    return m,  m*r

In [8]:
get_sample_size(0.10,0.11)

(14750.790469044954, 14750.790469044954)

### Вариант 5

In [9]:
# Разный размер выборок
get_sample_size(0.10,0.11,r=2)

(11124.279187859469, 22248.558375718938)

In [10]:
effect_size = sms.proportion_effectsize(baseline_rate, baseline_rate + practical_significance)
sample_size = sms.NormalIndPower().solve_power(effect_size = effect_size, power = sensitivity, 
                                               alpha = confidence_level, ratio=2)
print(sample_size)

11058.078627640636


### Вариант 6

In [11]:
def calc_sample_size(alpha, power, p, pct_mde, var='Absolute'):
    """ Based on https://www.evanmiller.org/ab-testing/sample-size.html

    Args:
        alpha (float): How often are you willing to accept a Type I error (false positive)?
        power (float): How often do you want to correctly detect a true positive (1-beta)?
        p (float): Base conversion rate
        pct_mde (float): Minimum detectable effect, relative to base conversion rate.

    """
    if var=='Absolute':
        delta = pct_mde
    else:
        delta = p*pct_mde

    t_alpha2 = norm.ppf(1.0-alpha/2)
    t_beta = norm.ppf(power)

    sd1 = np.sqrt(2 * p * (1.0 - p))
    sd2 = np.sqrt(p * (1.0 - p) + (p + delta) * (1.0 - p - delta))

    return (t_alpha2 * sd1 + t_beta * sd2) * (t_alpha2 * sd1 + t_beta * sd2) / (delta * delta)

In [12]:
calc_sample_size(0.05,0.8,0.1,0.01)

14312.856241916566

### Вариант 7

Публикация "Стратификация. Как разбиение выборки повышает чувствительность A/B теста", X5RetailGroup, Хабр. Авторы Николай Назаров, Александр Сахнов

Вероятности ошибок I и II рода положим равными 0.05 и 0.20 соответственно. Допустим, для пользователей, попадающих под условия эксперимента, на исторических данных мы получили оценку средней выручки (2500 рублей) и оценку её стандартного отклонения (800 рублей). После рассылки писем будем ожидать, что выручка увеличится минимум на 100 рублей.

In [13]:
alpha = 0.05                    # вероятность ошибки I рода
beta = 0.2                      # вероятность ошибки II рода
mu_control = 2500               # средняя выручка с пользователя в контрольной группе
delta = 0.04                    # размер эффекта (100 руб = 4 процента от 2500 руб )
std = 800                       # стандартное отклонение

mu_pilot = mu_control* (1 + delta) # средняя выручка с пользователя в экспериментальной группе
effect = mu_pilot - mu_control
t_alpha = stats.norm.ppf(1 - alpha / 2, loc=0, scale=1)
t_beta = stats.norm.ppf(1 - beta, loc=0, scale=1)
var = 2 * std ** 2
sample_size = int((t_alpha + t_beta) ** 2 * var / (effect ** 2))
print(f'sample_size = {sample_size}')

sample_size = 1004


### Вариант 8

In [14]:
sigma = 15 # стандартное отклонение
Z = 1.96 # квантиль нормального распределения.   
Delta = 5  # погрешность

# Расчет количества наблюдений для того, чтобы добиться точности 95%
N = int(((sigma*Z)/Delta)**2)
print(N)

34
